# Optimization Templates

In [154]:
# imports
import numpy as np
import sympy as sp

## Nonlinear Optimization

###  Function Definition and Derivatives

In [155]:
# manual function definition
def f_x(vec):
    x, y = np.array(vec).ravel()
    # return (x**4+y**4)
    return (x**2-2*x*y+x)**2

def grad_f(vec):
    x, y = np.array(vec).ravel()
    # return np.matrix([4*x**3, 4*y**3]).T
    return np.matrix([2*(x**2-2*x*y+x)*(2*x-2*y+1), -4*x*(x**2-2*x*y+x)]).T

def hessian_f(vec):
    x, y = np.array(vec).ravel()
    # return np.matrix([[12*x**2, 0], [0, 12*y**2]])
    return np.matrix([[4*(x**2-2*x*y+x)+2*(2*x-2*y+1)**2, -12*x**2-8*x+16*x*y], [-12*x**2-8*x+16*x*y, 8*x**2]])

In [156]:
# symbolic function definition
from sympy.abc import x,y,z
vars_f = [x,y]
# expr_f = x**4+y**4
# expr_f = (x**2-2*x*y+x)**2
# expr_f = (x-2)**4+(x-2*y)**2
# expr_f = (2*x-y**2)**2+(y-2-x)**2
# expr_f = (4*x**2-4*x*y+2*y**2)
expr_f = (1/2*x**2+(y/2-1)**2+2*x+2*y+6)

# vars_f = [x,y,z]
# expr_f = x**2 + y**2 + z**2

expr_grad = [sp.diff(expr_f, var) for var in vars_f]
expr_hessian = [[sp.diff(diff, var) for var in vars_f] for diff in expr_grad]

vars_f_str = ','.join([str(var) for var in vars_f])
print(f"f({vars_f_str}) = {expr_f}")
print()
print(f"∇f({vars_f_str}) = \n{np.matrix(expr_grad).T}")
print()
print(f"H_f({vars_f_str}) = \n{np.matrix(expr_hessian)}")

f_x_ = sp.lambdify(vars_f, expr_f)
grad_f_ = [sp.lambdify(vars_f, expr) for expr in expr_grad]
hessian_f_ = [[sp.lambdify(vars_f, expr) for expr in exprs] for exprs in expr_hessian]

def f_x(vec):
    return f_x_(*np.array(vec).ravel())

def grad_f(vec):
    return np.matrix([func(*np.array(vec).ravel()) for func in grad_f_]).T

def hessian_f(vec):
    return np.matrix([[func(*np.array(vec).ravel()) for func in funcs] for funcs in hessian_f_])

f(x,y) = 0.5*x**2 + 2*x + 2*y + (y/2 - 1)**2 + 6

∇f(x,y) = 
[[1.0*x + 2]
 [y/2 + 1]]

H_f(x,y) = 
[[1.00000000000000 0]
 [0 1/2]]


### Variable Definition

In [157]:
# initialize variables
x_0 = np.array(sp.Matrix([1,2]))

grad_0 = grad_f(x_0)
A_0 = hessian_f(x_0)

A_0_inv = np.matrix(sp.Matrix(A_0).inv())

print(f"f(x_0) = {f_x(x_0)}\n")
print(f"∇f(x_0) = \n{grad_0}\n")
print(f"A_0 = \n{A_0}\n")
print(f"A_0_inv = (rational/exact)\n{A_0_inv}\n\n{A_0_inv.astype(float)}")

f(x_0) = 12.5000000000000

∇f(x_0) = 
[[3.00000000000000]
 [2.00000000000000]]

A_0 = 
[[1.  0. ]
 [0.  0.5]]

A_0_inv = (rational/exact)
[[1.00000000000000 0]
 [0 2.00000000000000]]

[[1. 0.]
 [0. 2.]]


### Gradient Descent

In [158]:
def find_beta(x_0, beta_val_init=1, beta_exp_init=1):
    beta_val = beta_val_init
    beta_exp = beta_exp_init
    x_1 = x_0 - beta_val**beta_exp * grad_f(x_0)
    print("---------------------------------")
    print(f"β = 1/{beta_val} = {beta_val**beta_exp}\n")
    print(f"x_1:\n{x_1}\n")
    print(f"f(x_1) = {f_x(x_1)}\n")

    if f_x(x_1) >= f_x(x_0):
        beta_exp = -1
        while f_x(x_1) >= f_x(x_0):
            beta_val *= 2
            x_1 = x_0 - beta_val**beta_exp * grad_f(x_0)

            print("---------------------------------")
            print(f"β = 1/{beta_val} = {beta_val**beta_exp}\n")
            print(f"x_1:\n{x_1}\n")
            print(f"f(x_1) = {f_x(x_1)}\n")
    else:
        while f_x(x_1) < f_x(x_0):
            beta_val *= 2
            x_1_tmp = x_1
            x_1 = x_0 - beta_val**beta_exp * grad_f(x_0)

            if f_x(x_1) < f_x(x_0):
                print(f"β = {beta_val}\n")
                print(f"x_1:\n{x_1}\n")
                print(f"f(x_1) = {f_x(x_1)}\n")

        x_1 = x_1_tmp
    
    return beta_val, beta_exp

def fit_parabola(x_0, beta_val, beta_exp=1):
    beta = beta_val**beta_exp
    x_1 = x_0 - beta * grad_f(x_0)
    x_2 = x_0 - 2*beta * grad_f(x_0)

    print(f"P(0) = {f_x(x_0)}")
    print(f"P(β) = {f_x(x_1)}")
    print(f"P(2β) = {f_x(x_2)}\n")

    return beta/2*(3*f_x(x_0) - 4*f_x(x_1) + f_x(x_2))/(f_x(x_0) - 2*f_x(x_1) + f_x(x_2))

In [159]:
beta_val, beta_exp = find_beta(x_0, 1, 1)
x_1 = x_0 - beta_val**beta_exp * grad_f(x_0)

---------------------------------
β = 1/1 = 1

x_1:
[[-2.00000000000000]
 [0]]

f(x_1) = 5.00000000000000

β = 2

x_1:
[[-5.00000000000000]
 [-2.00000000000000]]

f(x_1) = 8.50000000000000



In [160]:
beta_star = fit_parabola(x_0, beta_val, beta_exp)
x_1_star = x_0 - beta_star * grad_f(x_0)

print(f"β* = {beta_star}\n")
print(f"x_1:\n{x_1_star}\n")
print(f"β : f(x_1) = {f_x(x_1)}")
print(f"β*: f(x_1) = {f_x(x_1_star)}")
print()

if f_x(x_1_star) < f_x(x_1):
    print(f"choosing β* = {beta_star}")
else:
    print(f"--> choosing β = {beta}")

P(0) = 12.5000000000000
P(β) = 48.5000000000000
P(2β) = 260.500000000000

β* = 1.18181818181818

x_1:
[[-2.54545454545455]
 [-0.363636363636364]]

β : f(x_1) = 48.5000000000000
β*: f(x_1) = 4.81818181818182

choosing β* = 1.18181818181818


### Newton's Method

In [161]:
def newton(A_inv, x, grad):
    return x - A_inv * grad

In [162]:
x_1 = newton(A_0_inv, x_0, grad_0)
print(f"x_1 = (rational/exact)\n{x_1}\n\n{x_1.astype(float)}\n")
print(f"f(x_1) = {float(f_x(x_1))}")

x_1 = (rational/exact)
[[-2.00000000000000]
 [-2.00000000000000]]

[[-2.]
 [-2.]]

f(x_1) = 4.0


### Broyden's Method

In [163]:
def broyden(A_inv, g, d):
    return A_inv - ((A_inv * g - d) * d.T * A_inv) / (d.T * A_inv *g)

In [164]:
# local broyden
_A_0_inv = np.matrix([[5,0],[0,2]])
_g = np.matrix([-1,1]).T
_d = np.matrix([-5,0]).T
broyden(_A_0_inv, _g, _d)


matrix([[5., 0.],
        [2., 2.]])

In [165]:
x_prev = x_0
A_inv_tmp = A_0_inv
for i in range(3):
    x_new = newton(A_inv_tmp, x_prev, grad_f(x_prev))
    print("---------------------------------")
    print(f"x_{i+1} = (rational/exact)\n{x_new}\n\n{x_new.astype(float)}\n")
    print(f"f(x_{i+1}) = {float(f_x(x_new))}\n")
    
    g = grad_f(x_new) - grad_f(x_prev)
    d = x_new - x_prev

    A_inv_tmp = broyden(A_inv_tmp, g, d)
    
    print(f"g_{i+1} = (rational/exact)\n{g}\n\n{g.astype(float)}\n")
    print(f"d_{i+1} = (rational/exact)\n{d}\n\n{d.astype(float)}\n")
    print(f"A_{i+1}_inv = (rational/exact)\n{A_inv_tmp}\n\n{A_inv_tmp.astype(float)}\n")
    
    x_prev = x_new

---------------------------------
x_1 = (rational/exact)
[[-2.00000000000000]
 [-2.00000000000000]]

[[-2.]
 [-2.]]

f(x_1) = 4.0

g_1 = (rational/exact)
[[-3.00000000000000]
 [-2.00000000000000]]

[[-3.]
 [-2.]]

d_1 = (rational/exact)
[[-3.00000000000000]
 [-4.00000000000000]]

[[-3.]
 [-4.]]

A_1_inv = (rational/exact)
[[1.00000000000000 0]
 [0 2.00000000000000]]

[[1. 0.]
 [0. 2.]]

---------------------------------
x_2 = (rational/exact)
[[-2.00000000000000]
 [-2.00000000000000]]

[[-2.]
 [-2.]]

f(x_2) = 4.0

g_2 = (rational/exact)
[[0]
 [0]]

[[0.]
 [0.]]

d_2 = (rational/exact)
[[0]
 [0]]

[[0.]
 [0.]]

A_2_inv = (rational/exact)
[[nan nan]
 [nan nan]]

[[nan nan]
 [nan nan]]

---------------------------------
x_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

f(x_3) = nan

g_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

d_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

A_3_inv = (rational/exact)
[[nan nan]
 [nan nan]]

[[nan nan]
 [nan nan]]



/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3602861214.py:16: RuntimeWarning: invalid value encountered in cast
  print(f"A_{i+1}_inv = (rational/exact)\n{A_inv_tmp}\n\n{A_inv_tmp.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3602861214.py:6: RuntimeWarning: invalid value encountered in cast
  print(f"x_{i+1} = (rational/exact)\n{x_new}\n\n{x_new.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3602861214.py:14: RuntimeWarning: invalid value encountered in cast
  print(f"g_{i+1} = (rational/exact)\n{g}\n\n{g.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3602861214.py:15: RuntimeWarning: invalid value encountered in cast
  print(f"d_{i+1} = (rational/exact)\n{d}\n\n{d.astype(float)}\n")


### Aitken's Acceleration Method

In [166]:
def aitken(x_0, x_1, x_2):
    return x_2 - np.divide(np.power(x_2 - x_1, 2), (x_2-2*x_1 + x_0))

In [167]:
print(f"y_2 = {aitken(100, 10, 2)}")
print(f"y_2 = {aitken(10, 2, .5)}")

y_2 = 1.2195121951219512
y_2 = 0.15384615384615385


In [168]:
x_prev = x_0
x_aitken = [x_0]
A_inv_tmp = A_0_inv
for i in range(3):
    x_new = newton(A_inv_tmp, x_prev, grad_f(x_prev))
    x_aitken.append(x_new)
    
    if len(x_aitken) >= 3:
        del x_aitken[:-3]
        x_new = aitken(*x_aitken)
    
    print("---------------------------------")
    print(f"x_{i+1} = (rational/exact)\n{x_new}\n\n{x_new.astype(float)}\n")
    print(f"f(x_{i+1}) = {float(f_x(x_new))}")
    
    g = grad_f(x_new) - grad_f(x_prev)
    d = x_new - x_prev

    A_inv_tmp = broyden(A_inv_tmp, g, d)
    
    print(f"g_{i+1} = (rational/exact)\n{g}\n\n{g.astype(float)}\n")
    print(f"d_{i+1} = (rational/exact)\n{d}\n\n{d.astype(float)}\n")
    print(f"A_{i+1}_inv = (rational/exact)\n{A_inv_tmp}\n\n{A_inv_tmp.astype(float)}\n")
    
    x_prev = x_new

---------------------------------
x_1 = (rational/exact)
[[-2.00000000000000]
 [-2.00000000000000]]

[[-2.]
 [-2.]]

f(x_1) = 4.0
g_1 = (rational/exact)
[[-3.00000000000000]
 [-2.00000000000000]]

[[-3.]
 [-2.]]

d_1 = (rational/exact)
[[-3.00000000000000]
 [-4.00000000000000]]

[[-3.]
 [-4.]]

A_1_inv = (rational/exact)
[[1.00000000000000 0]
 [0 2.00000000000000]]

[[1. 0.]
 [0. 2.]]

---------------------------------
x_2 = (rational/exact)
[[-2.00000000000000]
 [-2.00000000000000]]

[[-2.]
 [-2.]]

f(x_2) = 4.0
g_2 = (rational/exact)
[[0]
 [0]]

[[0.]
 [0.]]

d_2 = (rational/exact)
[[0]
 [0]]

[[0.]
 [0.]]

A_2_inv = (rational/exact)
[[nan nan]
 [nan nan]]

[[nan nan]
 [nan nan]]

---------------------------------
x_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

f(x_3) = nan
g_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

d_3 = (rational/exact)
[[nan]
 [nan]]

[[nan]
 [nan]]

A_3_inv = (rational/exact)
[[nan nan]
 [nan nan]]

[[nan nan]
 [nan nan]]



/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3787881117.py:23: RuntimeWarning: invalid value encountered in cast
  print(f"A_{i+1}_inv = (rational/exact)\n{A_inv_tmp}\n\n{A_inv_tmp.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3787881117.py:13: RuntimeWarning: invalid value encountered in cast
  print(f"x_{i+1} = (rational/exact)\n{x_new}\n\n{x_new.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3787881117.py:21: RuntimeWarning: invalid value encountered in cast
  print(f"g_{i+1} = (rational/exact)\n{g}\n\n{g.astype(float)}\n")
/var/folders/ft/p6xwtlq12ygfhmnytl1ngxdm0000gn/T/ipykernel_24815/3787881117.py:22: RuntimeWarning: invalid value encountered in cast
  print(f"d_{i+1} = (rational/exact)\n{d}\n\n{d.astype(float)}\n")


## Undirected Graph Algorithms

### Variable Definition

In [169]:
# adjacency matrix (undirected)
A = np.array(np.zeros((6,6)))
A[0,0:] = [0,200,580,np.nan,250,1200]
A[1,1:] = [0,500,820,np.nan,np.nan]
A[2,2:] = [0,230,150,1100]
A[3,3:] = [0,380,np.nan]
A[4,4:] = [0,np.nan]
A[5,5:] = [0]

for i in range(A.shape[0]):
    A[i:,i] = A[i,i:].T

n_vertices = A.shape[1]
vertices = ["A", "B", "C", "D", "E", "F"]

In [170]:
# adjacency matrix (undirected)
A = np.array(np.zeros((6,6)))
A[0,0:] = [0,60,160,np.nan,700,600]
A[1,1:] = [0,220,140,np.nan,800]
A[2,2:] = [0,20,900,500]
A[3,3:] = [0,800,300]
A[4,4:] = [0,40]
A[5,5:] = [0]

for i in range(A.shape[0]):
    A[i:,i] = A[i,i:].T

n_vertices = A.shape[1]
vertices = ["A", "B", "C", "D", "E", "F"]

In [171]:
# adjacency matrix (directed)
A = np.array(np.zeros((7,7)))
A[0] = [0,1,0,1,0,0,0]
A[1] = [0,0,1,0,0,1,0]
A[2] = [0,0,0,0,1,0,0]
A[3] = [0,1,0,0,0,0,0]
A[4] = [0,0,0,0,0,0,1]
A[5] = [1,0,0,1,0,0,0]
A[6] = [0,0,1,0,0,0,0]

A[A==0] = np.nan

n_vertices = A.shape[1]
vertices = ["A", "B", "C", "D", "E", "F", "G"]

In [172]:
# adjacency matrix (undirected)
A = np.array(np.zeros((6,6)))
A[0,0:] = [0,3,np.nan,6,9,np.nan]
A[1,1:] = [0,1,3,np.nan,2]
A[2,2:] = [0,1,4,4]
A[3,3:] = [0,2,np.nan]
A[4,4:] = [0,7]
A[5,5:] = [0]


for i in range(A.shape[0]):
    A[i:,i] = A[i,i:].T

n_vertices = A.shape[1]
vertices = ["1", "2", "3", "4", "5", "7"]

In [173]:
A

array([[ 0.,  3., nan,  6.,  9., nan],
       [ 3.,  0.,  1.,  3., nan,  2.],
       [nan,  1.,  0.,  1.,  4.,  4.],
       [ 6.,  3.,  1.,  0.,  2., nan],
       [ 9., nan,  4.,  2.,  0.,  7.],
       [nan,  2.,  4., nan,  7.,  0.]])

### Minimal Spanning Tree
#### Depth-First-Search

In [174]:
n_vertices = A.shape[1]

idx = 0
stack = []
output = []

while True:
    neighbors = np.arange(n_vertices, dtype=float)
    neighbors[np.isnan(A[idx])] = np.nan
    
    mask = np.zeros(len(neighbors), dtype=bool)
    mask[idx] = True
    mask[stack] = True
    mask[output] = True
    next_vertices = np.ma.MaskedArray(neighbors, mask)
    
    if np.any(np.isfinite(next_vertices)):
        stack.append(idx)
        idx = np.nanargmin(next_vertices)
        print(f"go to vertex {vertices[idx]}, stack: {[vertices[i] for i in stack]}")
    else:
        output.append(idx)
        if len(stack) > 0:
            idx = stack.pop(-1)
            print(f"iterate back to vertex {vertices[idx]}, stack: {[vertices[i] for i in stack]}")
        else:
            break

print()
print(f"indices : {output}")
print(f"sequence: {[vertices[idx] for idx in output]}")

go to vertex 2, stack: ['1']
go to vertex 3, stack: ['1', '2']
go to vertex 4, stack: ['1', '2', '3']
go to vertex 5, stack: ['1', '2', '3', '4']
go to vertex 7, stack: ['1', '2', '3', '4', '5']
iterate back to vertex 5, stack: ['1', '2', '3', '4']
iterate back to vertex 4, stack: ['1', '2', '3']
iterate back to vertex 3, stack: ['1', '2']
iterate back to vertex 2, stack: ['1']
iterate back to vertex 1, stack: []

indices : [5, 4, 3, 2, 1, 0]
sequence: ['7', '5', '4', '3', '2', '1']


#### Breath-Fisrt-Search

In [175]:
idx = 0
stack = []
output = []

while True:
    neighbors = np.arange(n_vertices, dtype=float)
    neighbors[np.isnan(A[idx])] = np.nan
    
    mask = np.zeros(len(neighbors), dtype=bool)
    mask[idx] = True
    mask[stack] = True
    mask[output] = True
    next_vertices = np.ma.MaskedArray(neighbors, mask)
    
    if np.any(np.isfinite(next_vertices)):
        stack.append(np.nanargmin(next_vertices))
        print(f"update stack: {[vertices[i] for i in stack]}")
    else:
        output.append(idx)
        if len(stack) > 0:
            idx = stack.pop(0)
            print(f"go to vertex {vertices[idx]}, stack: {[vertices[i] for i in stack]}")
        else:
            break

print()
print(f"indices : {output}")
print(f"sequence: {[vertices[idx] for idx in output]}")

update stack: ['2']
update stack: ['2', '4']
update stack: ['2', '4', '5']
go to vertex 2, stack: ['4', '5']
update stack: ['4', '5', '3']
update stack: ['4', '5', '3', '7']
go to vertex 4, stack: ['5', '3', '7']
go to vertex 5, stack: ['3', '7']
go to vertex 3, stack: ['7']
go to vertex 7, stack: []

indices : [0, 1, 3, 4, 2, 5]
sequence: ['1', '2', '4', '5', '3', '7']


### Prim's Algorithm

In [176]:
n_vertices = A.shape[1]

idx = 0
v_stack = [0]
e_stack = []

for i in range(20):
    mask = np.ones(A.shape, dtype=bool)
    for v_1 in v_stack:
        mask[v_1,:] = False
        for v_2 in v_stack:
            mask[v_1,v_2] = True

    next_vertices = np.ma.MaskedArray(A, mask)

    if np.any(np.isfinite(next_vertices)):
        stack.append(idx)
        e = np.sort(np.unravel_index(np.nanargmin(next_vertices), A.shape))
        
        for v in e:
            if v not in v_stack:
                v_stack.append(v)
        e_stack.append(e)
        print(f"selected edge: {vertices[e[0]]}{vertices[e[1]]}")
    else:
        break

weight = 0
for e in e_stack:
    weight += A[e[0],e[1]]

print(f"total minimal weight: {weight}")

selected edge: 12
selected edge: 23
selected edge: 34
selected edge: 27
selected edge: 45
total minimal weight: 9.0


### Dijkstra's Algorithm

In [177]:
V = [5] # start vertex
E = []
l = [0]

cnt = 1
print(f"{1:2d} | {' ':7s} | {str(V):30s} | {str(E):70s} | {l[-1]:6.1f}")

while True:
    mask = np.ones(A.shape, dtype=bool)
    for v_1 in V:
        mask[v_1,:] = False
        for v_2 in V:
            mask[v_1,v_2] = True

    next_vertices = np.ma.MaskedArray(A, mask)

    if np.any(np.isfinite(next_vertices)):
        e_stack = []
        l_stack = []
        for v_1, v_2 in zip(*np.where(np.logical_and(np.isfinite(A), ~mask))):
            e_stack.append([v_1,v_2])
            
            if v_1 in V:
                l_stack.append(l[V.index(v_1)] + A[v_1,v_2])
            elif v_2 in V:
                l_stack.append(l[V.index(v_2)] + A[v_1,v_2])
            
        next_idx = np.argmin(l_stack)
        e = e_stack[next_idx]
        e.sort()
        E.append(e_stack[next_idx])
        l.append(l_stack[next_idx])
        
        for v in e:
            if v not in V:
                V.append(v)
        
        print(f"{cnt+1:2d} | {str(e):7s} | {str(V):30s} | {str(E):70s} | {l[-1]:6.1f}")
        cnt += 1
    else:
        break

print()
print("Result\n-----------------")
for i in range(n_vertices):
    print(f"{i:2d} | {str(vertices[i]):3s} | {l[i]:6.1f}")

 1 |         | [5]                            | []                                                                     |    0.0
 2 | [1, 5]  | [5, 1]                         | [[1, 5]]                                                               |    2.0
 3 | [1, 2]  | [5, 1, 2]                      | [[1, 5], [1, 2]]                                                       |    3.0
 4 | [2, 3]  | [5, 1, 2, 3]                   | [[1, 5], [1, 2], [2, 3]]                                               |    4.0
 5 | [0, 1]  | [5, 1, 2, 3, 0]                | [[1, 5], [1, 2], [2, 3], [0, 1]]                                       |    5.0
 6 | [3, 4]  | [5, 1, 2, 3, 0, 4]             | [[1, 5], [1, 2], [2, 3], [0, 1], [3, 4]]                               |    6.0

Result
-----------------
 0 | 1   |    0.0
 1 | 2   |    2.0
 2 | 3   |    3.0
 3 | 4   |    4.0
 4 | 5   |    5.0
 5 | 7   |    6.0


### Floyd-Warshall Algorithm

In [178]:
def print_matrix(A, mask=None):
    if mask is None:
        mask = np.ones_like(A, dtype=bool)
    print("[", end="")
    for i in range(A.shape[0]):
        if i > 0:
            print(" ", end="")
            
        print("[", end="")
        for j in range(A.shape[0]):
            if mask[i,j]:
                print(f"{float(A[i,j]):6.1f}", end="")
            else:
                print("  ----", end="")
            
            if j < A.shape[1]-1:
                print(" ", end="")
        print("]", end="")
        if i < A.shape[0]-1:
            print()
    print("]",)

In [179]:
A_FW = A.copy()
A_FW[np.isnan(A_FW)] = np.inf

vertices = np.arange(A.shape[0])
# vertices = np.array([0,3,5])

for v in vertices:
    A_FW_tmp = A_FW.copy()
    
    for i in range(A_FW.shape[0]):
        for j in range(A_FW.shape[1]):
            if i == v or j == v:
                continue
            
            weight = A_FW[i,v] + A_FW[v,j]
            if weight < A_FW[i,j]:
                A_FW_tmp[i,j] = weight
                
    
    print(f"{v+1}.")
    print("changes:")
    print_matrix(A_FW_tmp, A_FW != A_FW_tmp)
    print("\nnew matrix:")
    print_matrix(A_FW_tmp)
    print()
    
    A_FW = A_FW_tmp

print("Result\n-----------------")
print("changes:")
print_matrix(A_FW, A_FW != A)


print("\nnew matrix:")
print_matrix(A_FW)

1.
changes:
[[  ----   ----   ----   ----   ----   ----]
 [  ----   ----   ----   ----   12.0   ----]
 [  ----   ----   ----   ----   ----   ----]
 [  ----   ----   ----   ----   ----   ----]
 [  ----   12.0   ----   ----   ----   ----]
 [  ----   ----   ----   ----   ----   ----]]

new matrix:
[[   0.0    3.0    inf    6.0    9.0    inf]
 [   3.0    0.0    1.0    3.0   12.0    2.0]
 [   inf    1.0    0.0    1.0    4.0    4.0]
 [   6.0    3.0    1.0    0.0    2.0    inf]
 [   9.0   12.0    4.0    2.0    0.0    7.0]
 [   inf    2.0    4.0    inf    7.0    0.0]]

2.
changes:
[[  ----   ----    4.0   ----   ----    5.0]
 [  ----   ----   ----   ----   ----   ----]
 [   4.0   ----   ----   ----   ----    3.0]
 [  ----   ----   ----   ----   ----    5.0]
 [  ----   ----   ----   ----   ----   ----]
 [   5.0   ----    3.0    5.0   ----   ----]]

new matrix:
[[   0.0    3.0    4.0    6.0    9.0    5.0]
 [   3.0    0.0    1.0    3.0   12.0    2.0]
 [   4.0    1.0    0.0    1.0    4.0    3.0]
 